# Part I. ETL Pipeline for Pre-Processing the Files

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
os.chdir('C:\\Users\\pit\\Google Drive\\Udacity\\Data_Modeling_with_Apache_Cassandra') #setting up for local development, please # it before run!
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

C:\Users\pit\Google Drive\Udacity\Data_Modeling_with_Apache_Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding.


#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [6]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

try:
    session.execute('''
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {'class':'SimpleStrategy', 'replication_factor':1}'''
    )
except Exception as e:
    print(e)


#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### Creating table

In [8]:
query = 'CREATE TABLE IF NOT EXISTS artist_by_session'
query = query + '(sessionId int,' \
                ' itemInSession int,' \
                ' artist text,' \
                ' song text,' \
                ' length float,' \
                ' PRIMARY KEY((sessionId, itemInSession)))'
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

#### Inserting values

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Notes:
For Query 1 where we need to filter by sessionId and itemInSession I've chosen the same columns
(sessionId and itemInSession) as PrimaryKey dur to it's uniqueness. Query doesn't require further filettering so no
clustering columns were used this time.

#### Check vs requirement

In [11]:
query = "select artist, song, length from artist_by_session WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


#### Creating table

In [21]:
query = 'CREATE TABLE IF NOT EXISTS users_by_session'
query = query + '(userId int,'\
                ' sessionId int,'\
                ' itemInSession int,'\
                ' artist text,'\
                ' song text,'\
                ' firstName text,'\
                ' lastName text,'\
                ' PRIMARY KEY((userId, sessionId), itemInSession))'
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Inserting values

In [22]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users_by_session (userId, sessionId, itemInSession, artist, song, firstName, lastName )"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4] ))

#### notes:
For Query 2 where we need to filter by userId and sessionId I've chosen the same columns
(userId and sessionId) as PrimaryKey however because requirement is to return song column in orderly manner therefore
song column were used as clustering column.

#### Check vs requirement

In [23]:
query = "select artist, song, firstName, lastName from users_by_session where userId = 10 and sessionId = 182;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)
                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Creating table

In [24]:
query = 'CREATE TABLE IF NOT EXISTS users_by_songs'
query = query + '(song text,' \
                ' userID int,' \
                ' firstName text,' \
                ' lastName text,'\
                ' PRIMARY KEY(song, userID))'
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Inserting values

In [25]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users_by_songs (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### notes:
For Query 3 where we need to return users listening to particular song I've chosen not only song column
but also to reference it to user - an unique userId column as PrimaryKey. None of columns were used as clustering columns
however adding lastName could result be returned in orderly manner.

#### Checking vs requirement

In [27]:
query = "select firstName, lastName from users_by_songs where song = 'All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.firstname, row.lastname)


Jacqueline Lynch
Tegan Levine
Sara Johnson


#### Dropping table and closing connection to db

In [29]:
#Drop the table before closing out the sessions
query = "DROP TABLE artist_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE users_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE users_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Close the session and cluster connection
session.shutdown()
cluster.shutdown()

('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9042 datacenter1>: ConnectionException('Pool is shutdown')})
('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9042 datacenter1>: ConnectionException('Pool is shutdown')})
('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9042 datacenter1>: ConnectionException('Pool is shutdown')})
